# <font color='blue'> Table Of Contents </font>

## <font color='blue'> Code Walkthrough: Mining and Validation </font>

* Introduction
* Proof-of-Work Based Mining
* Blockchain Validation
* Creation and Usage

# <font color='blue'> Code Walkthrough: Mining and Validation </font>

## <font color='blue'> Introduction </font>

We'll take the code developed in the last week's session and add basic versions of Proof-of-Work based mining and blockchain validation

The companion notebook 'M04W03-02-Basic-Blockchain-With-Mining-And-Validation-Source-Code' has the complete source code.


## <font color='blue'> Proof-of-Work Based Mining </font>

In [2]:
import time
import hashlib
import json

class Block:
    # Basic block init
    # Index: The position of the block in the chain starting with 0 for the genesis block
    # Timestamp: Time when the block was created
    # Transactions: Transactions included in the block
    # Previous block hash: Block hash of the previous block
    # Hash target: Block hash should be below this target
    # Nonce: Variable that miners vary to hit the hash target
    # Metadata: Any generic textual information added in the block (Optional)
    # Block hash: Hash of the current block including all the aforementioned data
    def __init__(self, index, transactions, previous_block_hash, hash_target, metadata=''):
        self._index = index
        self._timestamp = time.time()
        self._transactions = transactions
        self._previous_block_hash = previous_block_hash
        self._metadata = metadata
        self._hash_target = hash_target
        self._nonce = 0
        self._block_hash = None
        self.mine_block()

    def __str__(self):
        return f'\nBlock index: {self._index}\nTimestamp: {self._timestamp}\nTransactions: {self._transactions}\nPrevious Block Hash: {self._previous_block_hash}\nMetadata: {self._metadata}\nHash Target: {self._hash_target}\nNonce: {self._nonce}\nBlock Hash: {self._block_hash}\n'

    #__repr__ is called on the individual elements (instead of __str__) 
    # if you try to print a list of the items or objects
    def __repr__(self):
        return self.__str__()

    @property
    def block_hash(self):
        return self._block_hash

    @property
    def previous_block_hash(self):
        return self._previous_block_hash

    @property
    def hash_target(self):
        return self._hash_target

    # Serializing and utf-8 encoding relevant data, and then hashing and creating a hex representation
    def hash_block(self):
        hash_string = '-'.join([
                                str(self._index),
                                str(self._timestamp),
                                str(self._previous_block_hash),
                                str(self._metadata),
                                str(self._hash_target),
                                str(self._nonce),
                                json.dumps(self._transactions, sort_keys=True)
                                ])
        encoded_hash_string = hash_string.encode('utf-8')
        #print(f'Encoded string to be hashed for block {self._index}: {encoded_hash_string}\n\n')
        block_hash = hashlib.sha256(encoded_hash_string).hexdigest()
        return block_hash

    # Increasing nonce until block hash is below the hash target
    # Ignoring for genesis block (index 0) since that block is hard-coded
    def mine_block(self):
        if (self._index != 0):
            while (int(self.hash_block(), 16) > int(self._hash_target, 16)):
                self._nonce += 1
        self._block_hash = self.hash_block()
        return True


Here, we have added two new attributes:
1. Hash target: Hash target is a hexadecimal value which forms the outer limit of a valid block hash in the proof-of-work method. Since lower target values will automatically start having zeros at the beginning, it is also explained as having a specific number of zeros at the beginning. Doing the search using hex number comparison will be faster generally than finding the number of zeros at the front using string manipulation, and it's more generic.
2. Nonce: This is the attribute that a miner varies (generally starting from 0) to get an appropriate block hash.

You'll also notice that instead of just calculating and assigning the default block hash, we are now running a mine_block method in the constructor. Normally, this would be called separately after creating a candidate block. We have added it as part of the constructor itself, just for succinctness.

In the hash_block method, we have added both hash_target and nonce in the block hash generating string so that they are also unmodifiable later.

The mine_block method keeps comparing the block hash with the hash_target (hex comparison) for this block. If it's not less than the target, it'll increase the nonce and try again. Once it hits a particular value which is lower than the hash, it'll exit and use that nonce to store the final block hash. The number of iterations, on average, depend on the hash target value. The lower the target, the longer it'll take. The block is now ready for publishing.


## <font color='blue'> Blockchain Validation </font>

In [1]:
class Blockchain:
    # Basic blockchain init
    # Includes the chain as a list of blocks in order and pending transactions
    # Includes the current value of the hash target. It can be changed at any point to vary the difficulty
    # Also initiates a genesis block
    def __init__(self, hash_target):
        self._chain = []
        self._pending_transactions = []
        self._chain.append(self.__create_genesis_block())
        self._hash_target = hash_target

    def __str__(self):
        return f"Chain:\n{self._chain}\n\nPending Transactions: {self._pending_transactions}\n"

    @property
    def hash_target(self):
        return self._hash_target

    @hash_target.setter
    def hash_target(self, hash_target):
        self._hash_target = hash_target

    # Creating the genesis block, taking arbitrary previous block hash since there is no previous block
    # Using the famous bitcoin genesis block string here :)  
    def __create_genesis_block(self):
        genesis_block = Block(0, [], 'The Times 03/Jan/2009 Chancellor on brink of second bailout for banks', 
            None, 'Genesis block using same string as bitcoin!')
        return genesis_block

    # Creates a new block and appends to the chain
    # Also clears the pending transactions as they are part of the new block now
    def create_new_block(self):
        new_block = Block(len(self._chain), self._pending_transactions, self._chain[-1].block_hash, self._hash_target)
        self._chain.append(new_block)
        self._pending_transactions = []
        return new_block

    # Simple transaction with just one sender, one receiver, and one amount data
    def create_new_transaction(self, sender, receiver, amount, tx_metadata=''):
        new_transaction = {'sender': sender, 'receiver': receiver, 'amount': amount, 'tx_metadata': tx_metadata}
        self._pending_transactions.append(new_transaction)

    # Blockchain validation function
    # Validates that previous block hash stored in each block (except genesis) is equivalent to 
    # the actual hash of the previous block
    # Also validates that each block hash is lower than the block's hash target
    def validate_blockchain(self):
        for index in range(1, len(self._chain)):
            if (self._chain[index].previous_block_hash != self._chain[index - 1].hash_block()):
                print(f'Previous block hash mismatch in block index: {index}')
                return False

            if (int(self._chain[index].hash_block(), 16) >= int(self._chain[index].hash_target, 16)):
                print(f'Hash target not achieved in block index: {index}')
                return False
        return True



The only change, except for the validation function, is the addition of hash_target at the blockchain level. This will be set based on the current level of difficulty. It is passed to the Block constructor so that each block stores the target at the time it was created. The ongoing target value is regularly changed to ensure consistent throughput, depending on the number of miners, amount of computing power, etc.

validate_blockchain is a very basic validation function checking just two things:
1. Starting from the first non-genesis block, it checks whether the previous block hash value, saved in the current block, actually matches the calculated hash of the previous block data.
2. It calculates the hash of the block and ensures that it was set lower than the hash target of that block during mining. This will only be possible if the miner spent the resources to identify a nonce where this condition is true.


## <font color='blue'> Creation and Usage </font>

In [4]:
if __name__ == "__main__":
    block_chain = Blockchain('00ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff')
    time.sleep(1)

    block_chain.create_new_transaction('Alice', 'Bob', 20)
    block_chain.create_new_transaction('Bob', 'Carol', 30)
    block_chain.create_new_transaction('Carol', 'Alice', 50)
    block_chain.create_new_block()
    time.sleep(1)
    
    block_chain.hash_target = '0000ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'
    block_chain.create_new_transaction('Alice', 'Dave', 30)
    block_chain.create_new_transaction('Dave', 'Carol', 34)
    block_chain.create_new_transaction('Bob', 'Alice', 100)
    block_chain.create_new_block()
    time.sleep(1)

    block_chain.create_new_transaction('Alice', 'Carol', 46)
    block_chain.create_new_transaction('Bob', 'Dave', 90)
    
    print(block_chain)
    
    validation_result = block_chain.validate_blockchain()
    if (validation_result):
        print('Validation successful')
    else:
        print('Validation failed')


Chain:
[
Block index: 0
Timestamp: 1630136619.1754634
Transactions: []
Previous Block Hash: The Times 03/Jan/2009 Chancellor on brink of second bailout for banks
Metadata: Genesis block using same string as bitcoin!
Hash Target: None
Nonce: 0
Block Hash: 9cb8a184ac56891a54831b782ddeb9da5d2a7654525815cb648edbc00177bc13
, 
Block index: 1
Timestamp: 1630136620.1904697
Transactions: [{'sender': 'Alice', 'receiver': 'Bob', 'amount': 20, 'tx_metadata': ''}, {'sender': 'Bob', 'receiver': 'Carol', 'amount': 30, 'tx_metadata': ''}, {'sender': 'Carol', 'receiver': 'Alice', 'amount': 50, 'tx_metadata': ''}]
Previous Block Hash: 9cb8a184ac56891a54831b782ddeb9da5d2a7654525815cb648edbc00177bc13
Metadata: 
Hash Target: 00ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
Nonce: 25
Block Hash: 003748b4db8cb90e69dba135d6a2186fce0b1ce2f98a7e29e2db80e9d72c801f
, 
Block index: 2
Timestamp: 1630136621.2028787
Transactions: [{'sender': 'Alice', 'receiver': 'Dave', 'amount': 30, 'tx_metadata': ''

The initial hash target is set to '00ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'
You can see that it iterated the nonce value multiple times for block 1 before finding a block hash which is lower than the target. It has two zeros at the beginning.

Before block 2 creation, we increase the difficulty by reducing the hash target value. It now has four zeros at the beginning. You can compare the nonce value and see that it generally takes many more iterations now, and hence more computing power and time.

We also run the validation function which indicates that validation was successful. You can introduce appropriate errors in the code to make it fail.

Practical Exercise:
1. We are not validating transactions here in terms of amount validation. You can try to add that in the validation code. Starting from the genesis block, you'll have to keep track of the users and keep a running total, for each, to ensure that transactions are valid from that perspective.
The actual validation is much more complex primarily because each user's account is essentially a list of UTXOs which they can explicitly set as inputs in a transaction. We are also not signing and validating the signature as of now.

2. You'll notice that there is no block reward for mining and no specification for storing who mined a particular block. Add a miner argument while creating a block and store their name. Also, add an automatic transaction to give them a particular reward. This will be a different kind of transaction since cryptocurrency is being generated (equivalent to minting physical money) rather than being transferred from another sender.